In [1]:
import random
import re

import openai

from scapy.all import Raw
from scapy.utils import RawPcapReader, wrpcap

from scapy.layers.l2 import Ether
from scapy.layers.inet import IP, TCP, ICMP

In [2]:
sample_gen_num = 100

In [3]:
def process_pcap(file_name):
    print('Opening {}...'.format(file_name))

    count = 0
    interesting_packet_count = 0

    interesting_packets = []
    for (pkt_data, pkt_metadata,) in RawPcapReader(file_name):
        count += 1

        ether_pkt = Ether(pkt_data)
        if 'type' not in ether_pkt.fields:
            # LLC frames will have 'len' instead of 'type'.
            # We disregard those
            continue

        if ether_pkt.type != 0x0800:
            # disregard non-IPv4 packets
            continue
        ip_pkt = ether_pkt[IP]
        if ip_pkt.proto != 1:
            # Ignore non-TCP packet
            continue
        interesting_packet_count += 1
        interesting_packets.append(ether_pkt)

    print('{} contains {} packets ({} interesting)'.
          format(file_name, count, interesting_packet_count))

    return interesting_packets


In [4]:
packets_scapy = process_pcap("../pcap-analysis/data/encoder/pcap/custom_pings3.pcapng")

Opening ../pcap-analysis/data/encoder/pcap/custom_pings3.pcapng...
../pcap-analysis/data/encoder/pcap/custom_pings3.pcapng contains 2261 packets (2261 interesting)


In [5]:
len(packets_scapy)

2261

In [7]:
with open("ping_summaries.txt","r") as f:
    packets_summary = f.read().splitlines()

In [8]:
len(packets_summary)

2261

In [9]:
packets = tuple(zip(packets_summary,packets_scapy))

In [10]:
engines = openai.Engine.list()

In [43]:
responses = []
for i in range(sample_gen_num):
    summary,packet = random.choice(packets)
    packet = packet[IP]
    try:
        del packet[Raw]
    except IndexError:
        packet.show()
    del packet.chksum
    del packet[ICMP].chksum
    query = "This is the packet summary:\n"
    query += summary + "\n\n"
    query += "This is the packet details:\n"
    query += str(packet.show(dump=True))
    
    summary,_ = random.choice(packets)
    query += "\n\nThis is another packet summary:\n"
    query += summary + "\n\n"
    query += "Generate the packet details."
    
    print(query)
    break
    
    completion = openai.Completion.create(engine="text-davinci-003", prompt=query,max_tokens=3500)
    completion["prompt_summary"] = summary
    responses.append(completion)


This is the packet summary:
76.76.2.0 → 130.231.202.234 ICMP 98 Echo (ping) reply id=0x0005, seq=123/31488, ttl=48 (request in 1159)

This is the packet details:
###[ IP ]### 
  version   = 4
  ihl       = 5
  tos       = 0x0
  len       = 84
  id        = 50746
  flags     = 
  frag      = 0
  ttl       = 48
  proto     = icmp
  chksum    = None
  src       = 76.76.2.0
  dst       = 130.231.202.234
  \options   \
###[ ICMP ]### 
     type      = echo-reply
     code      = 0
     chksum    = None
     id        = 0x5
     seq       = 0x7b
     unused    = ''


This is another packet summary:
76.76.2.0 → 130.231.202.234 ICMP 98 Echo (ping) reply id=0x0005, seq=189/48384, ttl=48 (request in 1687)

Generate the packet details.


In [14]:
print(len(responses))

100


In [15]:
print(responses[0].prompt_summary)
print(responses[0].choices[0].text)

8.8.8.8 → 130.231.202.234 ICMP 98 Echo (ping) reply id=0x0002, seq=258/513, ttl=109 (request in 1881)
 

###[ IP ]### 
  version   = 4
  ihl       = 5
  tos       = 0x0
  len       = 84
  id        = 25911
  flags     = 
  frag      = 0
  ttl       = 109
  proto     = icmp
  chksum    = None
  src       = 8.8.8.8
  dst       = 130.231.202.234
  \options   \
###[ ICMP ]### 
     type      = echo-reply
     code      = 0
     chksum    = None
     id        = 0x2
     seq       = 0x102
     unused    = ''


In [39]:
def str_to_packet(s):
    packet = IP()
    s2i_proto = {v:k for k,v in packet.get_field("proto").i2s.items()}
    _,ip_text,payload_text = re.split("\#{3}\[\s*\w*\s*\]\#{3}",s)
    for line in ip_text.splitlines():
        if line == " " or line == "  \\options   \\":
            continue

        try: 
            key,value = line.split("=")
        except ValueError:
            print(line)
            continue
        if key.strip() == "proto":
            packet[IP].proto = s2i_proto[value.strip()]
            continue
        try:
            packet[IP].fields[key.strip()] = int(value.strip(),0)
        except:
            
            packet[IP].fields[key.strip()] = value.strip()
    # Handle ICMP Payload 
    if packet[IP].proto == 1:
        packet = packet / ICMP()
        type_conversion = {"echo-reply" : 0, "echo-request":8}
        for line in payload_text.splitlines():
            if line == " " or line == "  \\options   \\":
                continue
                
            try: 
                key,value = line.split("=")
            except ValueError:
                print(line)
                continue
            
            try:
                packet[ICMP].fields[key.strip()] = int(value.strip(),0)
            except:
                packet[ICMP].fields[key.strip()] = value.strip()

        if packet.unused == "''":
            packet.unused = b''
        packet.type = type_conversion[packet.type]
    return packet

In [40]:
gen_pkts = [str_to_packet(response.choices[0].text) for response in responses]

def delete_wrong_fields(pkt):

    del pkt[IP].flags
    del pkt[IP].chksum
    del pkt[ICMP].chksum
    
    return pkt
    
gen_pkts = list(map(delete_wrong_fields,gen_pkts))
gen_pkts[0].show2()

with open("generated_icmp.pcap", "wb") as f:
    wrpcap(f, gen_pkts)






  









###[ IP ]### 
  version   = 4
  ihl       = 5
  tos       = 0x0
  len       = 84
  id        = 25911
  flags     = 
  frag      = 0
  ttl       = 109
  proto     = icmp
  chksum    = 0x8a90
  src       = 8.8.8.8
  dst       = 130.231.202.234
  \options   \
###[ ICMP ]### 
     type      = echo-reply
     code      = 0
     chksum    = 0xfefb
     id        = 0x2
     seq       = 0x102
     unused    = ''



In [41]:
len(gen_pkts)

100

In [24]:
print(responses[0].choices[0].text)



###[ IP ]###
  version   = 4
  ihl       = 5
  tos       = 0x0
  len       = 84
  id        = 22238
  flags     = DF
  frag      = 0
  ttl       = 64
  proto     = icmp
  chksum    = None
  src       = 130.231.202.234
  dst       = 8.8.8.8
  \options   \
###[ ICMP ]###
     type      = echo-request
     code      = 0
     chksum    = None
     id        = 0x2
     seq       = 0xb2
     unused    = ''


In [173]:
summary,packet = random.choice(packets)

In [42]:
summary

'76.76.2.0 → 130.231.202.234 ICMP 98 Echo (ping) reply id=0x0005, seq=37/9472, ttl=48 (request in 471)'

In [160]:
packet.show()

###[ Ethernet ]### 
  dst       = 84:a9:38:6a:7f:0d
  src       = 70:db:98:81:93:40
  type      = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 84
     id        = 53490
     flags     = 
     frag      = 0
     ttl       = 53
     proto     = icmp
     chksum    = 0x54d3
     src       = 9.9.9.9
     dst       = 130.231.202.234
     \options   \
###[ ICMP ]### 
        type      = echo-reply
        code      = 0
        chksum    = 0x3c6
        id        = 0x4
        seq       = 0x101
        unused    = ''
###[ Raw ]### 
           load      = '\\xd5I\\x87c\x00\x00\x00\x00Ӵ\x0c\x00\x00\x00\x00\x00\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a\x1b\x1c\x1d\x1e\x1f !"#$%&\'()*+,-./01234567'



In [156]:
type(gen_pkts[0][ICMP].unused) == type(packet[ICMP].unused)

True

In [167]:
gen_pkts[2][IP].flags

<Flag 0 ()>

In [168]:
packet[IP].flags

<Flag 0 ()>

In [112]:
gen_pkts[0].show()

###[ IP ]### 
  version   = 4
  ihl       = 5
  tos       = 0x0
  len       = 84
  id        = 49001
  flags     = DF
  frag      = 0
  ttl       = 64
  proto     = icmp
  chksum    = None
  src       = 130.231.202.234
  dst       = 8.8.8.8
  \options   \
###[ ICMP ]### 
     type      = echo-request
     code      = 0
     chksum    = None
     id        = 0x2
     seq       = 0xb8
     unused    = ''



In [186]:
summary,packet2 = random.choice(packets)
packet2[IP].show()
packet2[IP].flags = "DF"
del packet2[Raw]
del packet2[IP].chksum
del packet2[ICMP].chksum
packet2[IP].show()


###[ IP ]### 
  version   = 4
  ihl       = 5
  tos       = 0x0
  len       = 84
  id        = 14963
  flags     = 
  frag      = 0
  ttl       = 49
  proto     = icmp
  chksum    = 0xfb5e
  src       = 4.2.2.4
  dst       = 130.231.202.234
  \options   \
###[ ICMP ]### 
     type      = echo-reply
     code      = 0
     chksum    = 0xf79f
     id        = 0x3
     seq       = 0x62
     unused    = ''
###[ Raw ]### 
        load      = '"I\\x87c\x00\x00\x00\x00\\x96{\t\x00\x00\x00\x00\x00\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a\x1b\x1c\x1d\x1e\x1f !"#$%&\'()*+,-./01234567'

###[ IP ]### 
  version   = 4
  ihl       = 5
  tos       = 0x0
  len       = 84
  id        = 14963
  flags     = DF
  frag      = 0
  ttl       = 49
  proto     = icmp
  chksum    = None
  src       = 4.2.2.4
  dst       = 130.231.202.234
  \options   \
###[ ICMP ]### 
     type      = echo-reply
     code      = 0
     chksum    = None
     id        = 0x3
     seq       = 0x62
     unused    = ''



In [184]:
packet2[IP].flags

<Flag 2 (DF)>

In [187]:
with open("generated_icmp.pcap", "wb") as f:
    wrpcap(f, packet2)
